 <div style="text-align:center;">
  <span style="color:green; font-size:2em; font-weight:bold;">Modélisation</span><br><br>

# <span style="color:green">Introduction</span><br><br> 

### <span style="color:green">Références et équations de base de la relation d'Okun</span><br><br> 

Dans cette partie, nous allons proposer une modélisation permettant de vérifier empiriquement la loi d'Okun, à partir de la base de données "final_data", établie dans la partie "Préparation_données.ipynb".

La loi d'Okun, indique que : "chaque fois que le PIB chute d'un certain pourcentage, par rapport au produit potentiel, le taux de chômage augmente d'environ un point de pourcentage. Ainsi, lorsque le PIB réel diminue, le taux de chômage augmente." (Mankiw, 2003)

Pour cette modélisation, on s'inspire des travaux suivants : 

- [1]. Traoré, D. L., Diakite, S., & Mariko, O. (2021). Croissance et chômage au Mali : Vérification empirique de la loi d'Okun. Revue Malienne de Science et de Technologie, Série C : Sciences Humaines et Sociales, 02(25), Page. ISSN 1987-1031. CNRST, Bamako, Mali.

- [2]. Aassif, Z. L'extraction de la tendance cycle. Haut-Commissariat au Plan, Direction de la comptabilité nationale. 

- [3]. Mills, T. C. (2019). Chapter 12 - Transfer Functions and Autoregressive Distributed Lag Modeling. In Applied Time Series Analysis: A Practical Guide to Modeling and Forecasting (pp. 201-210). Loughborough University, Loughborough, United Kingdom. https://doi.org/10.1016/B978-0-12-813117-6.00012-0

Le travail [1] nous présente deux équations de base de la relation d'Okun, la version en "différences premières" et la "version gap".

- Version en "différences premières" : 
$$\Delta u = c + \beta\Delta \text{pib} + \epsilon$$

- "Version gap" : 
$$u - u^* = c + \beta^*(\text{pib} - \text{pib}^*) + \epsilon$$

avec $u$ le taux de chômage, $u^*$ le taux de chômage naturel, $\text{pib}^*$ le PIB potentiel et c, $\beta$ et $\epsilon$  des constantes à déterminer grâce à la modélisation. 

### <span style="color:green">Équation et outils utilisés dans notre modélisation</span><br><br> 

Pour la modélisation, nous allons utiliser la version "gap", qui relie le taux de chômage et le PIB de la manière suivante : 
**$$u - u^* = c + \beta^*(\text{pib} - \text{pib}^*) + \epsilon$$**

Pour extraire la tendance à long terme du taux de chômage et du PIB (c'est-à-dire, le taux de chômage naturel et le PIB potentiel), à partir de la base de données, nous allons utiliser le filtre de **Hodrick-Prescott**. Ce filtre est introduit dans [2], et permet de décomposer une série selon sa tendance à long terme et son cycle (fluctuations autour de la tendance) à plus court terme.

Par ailleurs, pour prendre en compte la dynamique temporelle de nos données, nous allons utiliser le modèle **ARDL** (Auto Regressive Distributed Lag), introduit dans [2].

### <span style="color:green">Les différentes étapes de notre modélisation</span><br><br> 


**1. Clustering des Pays :**
Dans cette première étape, nous regroupons les pays présents dans la base de données selon des critères socio-économiques à l'aide de la technique du **clustering**.

**2. Sélection Aléatoire d'un pays par groupe :**
Au sein de chaque groupe formé lors du clustering, nous choisissons aléatoirement un pays afin d'assurer une représentation équitable.

**3. Modélisation de la Loi d'Okun avec ARDL :**
Nous allons utiliser le modèle ARDL pour modéliser la relation entre le taux de chômage et le PIB réel, et ce pour chacun des pays sélectionnés à l'étape précédente. La modélisation se présente les étapes suivantes :

- Prétraitement des séries temporelles :  nous commençons par différencier les séries des "gap" (écarts entre le taux de chômage naturel et réel, et entre le PIB potentiel et réel) jusqu'à ce qu'elles deviennent stationnaires. La stationnarité est cruciale pour assurer la stabilité des relations à long terme.

- Détermination des paramètres Lags : les paramètres lags nécessaires pour le modèle ARDL sont déterminés en utilisant les fonctions d'autocorrélation (ACF) et d'autocorrélation partielle (PACF). Ces fonctions aident à identifier les retards significatifs à prendre en compte dans la modélisation.

- Régression OLS : Enfin, une régression OLS (Ordinary Least Squares) est appliquée en tenant compte des paramètres lags déterminés. Cela permet de modéliser la relation entre le taux de chômage et l'écart entre le PIB potentiel et le PIB réel, en prenant en compte la dynamique temporelle. L'utilisation de la régression OLS dans le contexte de l'ARDL simplifie la modélisation tout en permettant une interprétation claire des paramètres.

# <span style="color:green"> I- Préparation des Dataframes</span><br><br> 

In [ ]:
# Import nécessaire
import declarations as d 

In [ ]:
# Importation de la base de données obtenue suite au nettoyage
data = d.pd.read_csv('bases/final_data.csv', encoding='latin-1')

In [ ]:
# Visualisation
data.head(2)

### <span style="color:green"> Base de données pour la modélisation</span><br><br> 

Pour la modélisation de la relation entre le PIB et le taux de chômage, nous aurons besoin de ces deux variables pour chaque pays en colonnes et des dates en indice. 

In [ ]:
# Utilisation de la fonction pivot pour remodeler le dataframe
df_pivot = data.pivot(index=['YEAR'], columns='COUNTRY',
                    values=['Unemployment_rate', 'GDP_rate'])

# Ajuster les noms de colonnes
df_pivot.columns = [f'{col[1]}_{col[0].lower()}' for col in df_pivot.columns]

# Réinitialiser l'index pour avoir les colonnes YEAR et MONTH comme des colonnes régulières
df_pivot.reset_index(inplace=True)

# Renommer les colonnes (supprimer le suffixe _rate)
df_pivot = df_pivot.rename(columns=lambda x: x.replace('_rate', ''))

In [ ]:
# Visualisation du nouveau dataframe
df_pivot.head(2)

### <span style="color:green"> Base de données pour le clustering </span><br><br> 

Pour le clustering, il nous faut une base de données avec en indice le nom des pays et en colonnes, les différents critères considérés, moyennés sur la période considérée.

In [ ]:
# Utilisation de la fonction pivot pour remodeler le dataframe
df_cluster = data.pivot_table(index='COUNTRY', values=['Unemployment_rate', 'GDP_rate','life_expentancy',
       'pop_growth_rate'], aggfunc='mean')

In [ ]:
# Visualisation
df_cluster.head(2)

# <span style="color:green">II- Clustering : regroupement des pays selon des critères socio-économiques</span><br><br> 

Dans cette partie, le clustering nous permettra de tirer des groupes homogènes de pays. Quatres (4) critères sont retenus : 

<div style="margin-left: 20px;">
    <span style="font-weight:bold;">1.</span> Le PIB,<br>
    <span style="font-weight:bold;">2.</span> le taux de chômage,<br>
    <span style="font-weight:bold;">3.</span> l'espérance de vie, <br>
    <span style="font-weight:bold;">4.</span> et le taux de croissance de la population.
    </div>

Chaque variable étant moyennée sur la période considérée. 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Supprimer les lignes ayant des valeurs manquantes
df_cluster.dropna(inplace=True)

In [ ]:
# Visualisation 
df_cluster.head()

In [ ]:
# Standardiser les données (important pour K-means)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Appliquer l'algorithme K-means
num_clusters = 4 
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_cluster['Cluster'] = kmeans.fit_predict(df_scaled)

# Visualisation en 3D (utilisant les variables GDP_rate, Life_expectance, Pop_growth_rate)
fig = d.plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_scaled[:, 1], df_scaled[:, 2], df_scaled[:, 3], c=df_cluster['Cluster'], cmap='viridis', s=50)
ax.set_xlabel('GDP_rate')
ax.set_ylabel('Life_expectancy')
ax.set_zlabel('Pop_growth_rate')
d.plt.show()

In [ ]:
# Afficher les groupes et les pays
for cluster_num in range(num_clusters):
    countries_in_cluster = df_cluster[df_cluster['Cluster'] == cluster_num].index
    print(f'Cluster {cluster_num + 1}: {list(countries_in_cluster)}')

# <span style="color:green">III- Extraction de la tendance du taux de chômage et du PIB</span><br><br> 

## <span style="color:green">III-1. Base de données du taux de chômage pour chaque pays </span><br><br> 

In [ ]:
# Extraire toutes les colonnes avec le suffixe "_rate" ainsi que la colonne des dates
columns_unemployment = ['YEAR'] + [col for col in df_pivot.columns if col.endswith('_unemployment')] 
df_unemployment = df_pivot[columns_unemployment]

# Définir la colonne 'YEAR' comme index 
df_unemployment.set_index('YEAR', inplace = True)

In [ ]:
# Visualisation
df_unemployment.head(2)

## <span style="color:green">III-2. Base de données du PIB réel pour chaque pays</span><br><br> 

In [ ]:
# Extraire toutes les colonnes avec le suffixe "_gdp" ainsi que la colonne des dates
columns_gdp = ['YEAR'] + [col for col in df_pivot.columns if col.endswith('_gdp')] 
df_gdp = df_pivot[columns_gdp]

# Définir la colonne 'YEAR' comme index 
df_gdp.set_index('YEAR', inplace = True)

In [ ]:
# Visualisation
df_gdp.head(2)

## <span style="color:green">III-3. Extraction de la tendance du PIB et du taux de chômage d'un pays </span><br><br> 


Dans cette partie, on définit une classe qui permet de tracer la tendance à long terme et les cycles à court terme des deux variables considérées. 

La méthode **df_gap()** de cette classe permet d'obtenir une série avec les valeurs de la différence entre la valeur réelle et la tendance des deux variables considérées, pour un pays considéré. 

In [ ]:
class CountryData:
    def __init__(self, country_code, unemployment_data, gdp_data):
        """
        Initialize a CountryData instance.

        Parameters:
        - country_code (str): The country code.
        - unemployment_data (pd.Series): Time series data for unemployment rate.
        - gdp_data (pd.Series): Time series data for GDP rate.
        """
        
        self.country_code = country_code
        self.unemployment_rate = unemployment_data[f'{country_code}_unemployment']
        self.gdp_rate = gdp_data[f'{country_code}_gdp']

    def plot_trend_cycle(self):
        """
        Plot the trend and cycle components for unemployment rate and GDP rate after applying the Hodrick-Prescott filter to the time series.
        """
        # Extraction of the long-term trend of unemployment rate
        x = self.unemployment_rate

        # Applying the Hodrick-Prescott filter
        trend_x, cycle_x = d.smf.hpfilter(x, lamb=1600)

        # Plot the trend and cycle of unemployment rate
        d.plt.plot(x, label='Original series')
        d.plt.plot(trend_x, label='Trend')
        d.plt.plot(cycle_x, label='Cycle')
        d.plt.legend()
        d.plt.show()

        # Extraction de la tendance à long terme du PIB
        y = self.gdp_rate

        # Appliquer le filtre de Hodrick-Prescott 
        trend_y, cycle_y = d.smf.hpfilter(y, lamb=1600)

        # Plot the trend and cycle of the trend
        d.plt.plot(y, label='Originale series')
        d.plt.plot(trend_y, label='Trend')
        d.plt.plot(cycle_y, label='Cycle')
        d.plt.legend()
        d.plt.show()
        
    def df_gap(self):
        
        """
        Calculate the gap series for unemployment rate and GDP rate.

        Returns:
        - df_gap (pd.DataFrame): DataFrame containing the gap values.
        """
        # Extraction of the long-term trend of unemployment rate
        x = self.unemployment_rate

        # Applying the Hodrick-Prescott filter
        trend_x, cycle_x = d.smf.hpfilter(x, lamb=1600)
    
        # Extraction of the long-term trend of GDP
        y = self.gdp_rate

        # Applying the Hodrick-Prescott filter
        trend_y, cycle_y = d.smf.hpfilter(y, lamb=1600)
    
        # Calculate the new series by removing the trend
        gap_unemployment = x - trend_x
        gap_gdp = y - trend_y

        # Create a DataFrame with the new series
        df_x = d.pd.DataFrame({'YEAR': x.index, 'gap_unemployment': gap_unemployment.values})
        df_y = d.pd.DataFrame({'YEAR': y.index, 'gap_gdp': gap_gdp.values})

        # Merge the DataFrames on the 'YEAR' column
        df_gap = d.pd.merge(df_x, df_y, on='YEAR', how='inner')

        return df_gap


# <span style="color:green">IV- Vérification de la loi pour un pays tiré aléatoirement dans chaque groupe</span><br><br>

## <span style="color:green">IV-1. Modèle ARDL : modélisation de la relation PIB/taux de chômage </span><br><br>

In [ ]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
class PreProcessing:
    def __init__(self, country_code, unemployment_data, gdp_data):
        self.country_code = country_code
        self.unemployment_rate = unemployment_data[f'{country_code}_unemployment']
        self.gdp_rate = gdp_data[f'{country_code}_gdp']
        self.country = CountryData(self.country_code, unemployment_data, gdp_data)
        self.unemployment_series = self.country.df_gap()['gap_unemployment']
        self.gdp_series = self.country.df_gap()['gap_gdp']

    def differentiation(self, series):
        # Différenciation des séries temporelles
        return series.diff().dropna()

    def stationarity_test(self, series, name):
        result = d.adfuller(series, autolag='AIC')
        #print(f'Test ADF pour la série {name}:')
        #print(f'ADF Statistic: {result[0]}')
        #print(f'p-value: {result[1]}')
        #print(f'Nombre de lags utilisés: {result[2]}')
        #print(f'Nombre d\'observations utilisées: {result[3]}')
        #print(f'Valeurs critiques:')
        #for key, value in result[4].items():
            #print(f'   {key}: {value}')

        p_value = result[1]
        i = 0
        diff_series = [series]
        while p_value > 0.05:
            diff_series.append(self.differentiation(diff_series[i]))
            result = d.adfuller(diff_series[i + 1], autolag='AIC')
            p_value = result[1]
            i += 1

        return diff_series[i]

    def ACF_PACF_plot(self):
        """Cette fonction permet d'afficher les graphiques de l'ACF et du PACF des séries stationnaires (obtenues après différenciation ou non).
        A partir de ces graphiques, nous allons pouvoir déterminer le nombre de retards significatifs à considérer pour chacune des séries."""
        
        diff_unemployment_series = self.stationarity_test(self.unemployment_series, 'Unemployment_rate')
        diff_gdp_series = self.stationarity_test(self.gdp_series, 'Real_GDP')

        df_diff = d.pd.DataFrame({
            'Unemployment_rate': diff_unemployment_series,
            'Real_GDP': diff_gdp_series
        })

        # Créer une figure et des axes pour les graphiques
        fig, axes = d.plt.subplots(nrows=2, ncols=2, figsize=(16, 12))

        # ACF - Diff_Unemployment_rate
        plot_acf(df_diff['Unemployment_rate'], ax=axes[0, 0], lags=40, title='ACF - Unemployment_rate')

        # PACF - Diff_Unemployment_rate
        plot_pacf(df_diff['Unemployment_rate'], ax=axes[0, 1], lags=40, title='PACF - Unemployment_rate')

        # ACF - Diff_Real_GDP
        plot_acf(df_diff['Real_GDP'], ax=axes[1, 0], lags=40, title='ACF - Real_GDP')

        # PACF - Diff_Real_GDP
        plot_pacf(df_diff['Real_GDP'], ax=axes[1, 1], lags=40, title='PACF - Real_GDP')

        # Ajuster l'espacement entre les graphiques
        d.plt.tight_layout()

        # Afficher la figure
        d.plt.show()

In [ ]:
class ARDLAnalysis:
    def __init__(self, country_code, unemployment_data, gdp_data, lags_ur, lags_gdp):
        self.country_code = country_code
        self.lags_ur = lags_ur
        self.lags_gdp = lags_gdp
        self.unemployment_series, self.gdp_series = self.preprocess_data(unemployment_data, gdp_data)

    def preprocess_data(self, unemployment_data, gdp_data):
        preprocessing = PreProcessing(self.country_code, unemployment_data, gdp_data)
        return preprocessing.stationarity_test(preprocessing.unemployment_series, 'Unemployment_rate'), \
               preprocessing.stationarity_test(preprocessing.gdp_series, 'Real_GDP')

    def create_explanatory_matrix(self):
        df_diff = d.pd.DataFrame({
            'Unemployment_rate': self.unemployment_series,
            'Real_GDP': self.gdp_series
        })
        self.df_diff = df_diff 
        x = sm.add_constant(df_diff[['Unemployment_rate', 'Real_GDP']])
        for lag in range(1, max(self.lags_ur, self.lags_gdp) + 1):
            x[f'Unemployment_rate_{lag}'] = df_diff['Unemployment_rate'].shift(lag)
            x[f'Real_GDP_{lag}'] = df_diff['Real_GDP'].shift(lag)
        return x.dropna()

    def define_dependent_variable(self):
        y = self.df_diff['Real_GDP'].shift(-1).dropna()
        return y[y.index.isin(self.x.index)]

    def fit_ardl_model(self):
        self.x = self.create_explanatory_matrix()
        self.y = self.define_dependent_variable()

        # Vérifier que les deux dataframes ont bien les mêmes données
        Y = self.y[self.y.index.isin(self.x.index)]
        X = self.x[self.x.index.isin(Y.index)]

        # Ajuster le modèle ARDL
        model = sm.OLS(Y, X)
        results = model.fit()

        # Afficher les résultats
        return results.summary()

## <span style="color:green">IV-2.  Application du modèle à un pays sélectionné aléatoirement pour chaque cluster </span><br><br>

### <span style="color:green">IV.2.1. Tirage aléatoire d'un pays dans chaque cluster </span><br><br>

In [ ]:
# Afficher le pays sélectionné de manière aléatoire dans chaque cluster
list_random_countries = []
for cluster_num in range(num_clusters):
    countries_in_cluster = df_cluster[df_cluster['Cluster'] == cluster_num].index
    random_country = d.np.random.choice(countries_in_cluster)
    list_random_countries.append(random_country)
    print(f'Cluster {cluster_num + 1}: {random_country}')

In [ ]:
print(f'Le code des pays sélectionnés dans chaque cluster sont les suivants', list_random_countries)

### <span style="color:green">IV.2.2. Pays du cluster 1  </span><br><br>

La stationnarité est une condition importante pour l'application de modèles ARDL. Cette première étape va donc permettre de vérifier si les séries Unemployment_rate et Real_GDP sont stationnaires et de les différencier jusqu'à l'obtention de séries stationnaires, le cas contraire.

Unemployment_rate contient les valeurs de la différence entre le taux de chômage réel et sa tendance.
Real_GDP contient les valeurs de la différence entre le PIB réel et sa tendance.

Par ailleurs, après obtention de séries stationnaires, nous souhaitons déterminer le nombre approprié de retards significatifs (lags) à considérer dans notre modèle pour chacune des séries. Pour cela, nous pouvons nous servir des graphiques de la fonction d'autocorrélation (ACF) et de la fonction d'autocorrélation partielle (PACF). Ces graphiques aident à visualiser les corrélations entre les observations à différents retards. Nous devons rechercher des retards significatifs où les corrélations sont en dehors de l'intervalle de confiance.

In [ ]:
country_code1 = list_random_countries[0]
country1 = PreProcessing(country_code1, df_unemployment, df_gdp)
country1.ACF_PACF_plot()

En observant les graphiques, le lecteur doit définir les lags appropriés dans la cellule code qui suit. Pour cela, il faut compter le nombre de pics en dehors de l'intervalle de confiance des graphes ACF et PACF et retenir le nombre maximal entre les deux, et ce, pour chacune des variables "Unemployment_rate" et "Real_GDP".

N.B. Si l'ACF et le PACF sont nuls sauf pour un nombre fini de points, il faut prendre un nombre de lags nul.

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_1 = 6  # Lags pour Unemployment_rate
lags_gdp_1 = 0  # Lags pour Real_GDP

La cellule code qui suit, nous permet d'obtenir les résultats de la modélisation. 

N.B. Le modèle utilisé est une spécification simple d'un modèle ARDL (AutoRegressive Distributed Lag), qui utilise la régression OLS. Cependant, la dynamique temporelle est partiellement prise en compte dans la mesure où le modèle inclut un lag du taux de chômage (Unemployment_rate) pour capturer l'effet retardé sur la variable dépendante (Real_GDP).

In [ ]:
country1_model = ARDLAnalysis(country_code1, df_unemployment, df_gdp, lags_ur_1, lags_gdp_1)
country1_model.fit_ardl_model()

Conclusion : 

Nous réitérons la même méthode pour les autres pays sélectionnés.

### <span style="color:green">IV.2.3. Pays du cluster 2  </span><br><br>

In [ ]:
country_code2 = list_random_countries[1]
country2 = PreProcessing(country_code2, df_unemployment, df_gdp)
country2.ACF_PACF_plot()

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_2 = 6  # Lags pour Unemployment_rate
lags_gdp_2 = 5  # Lags pour Real_GDP

In [ ]:
# Exemple avec AutoReg
from statsmodels.tsa.api import AutoReg

In [ ]:
import pandas as pd
from statsmodels.tsa.api import AutoReg
from statsmodels.tools import add_constant


test = ARDLAnalysis(country_code2, df_unemployment, df_gdp, lags_ur_2, lags_gdp_2)
data_diff = d.pd.DataFrame({
            'Unemployment_rate': test.unemployment_series,
            'Real_GDP': test.gdp_series
        })

# Ajoute une constante à la matrice explicative
data_diff_with_const = add_constant(data_diff)

# Spécifie le modèle AutoReg avec les retards
lags_ur = 1  # ajuste le nombre de lags pour le taux de chômage
lags_gdp = 1  # ajuste le nombre de lags pour le PIB réel
model = AutoReg(data_diff['Diff_Real_GDP'], lags=[lags_ur, lags_gdp], trend='c', exog=data_diff_with_const)
results = model.fit()

# Affiche les résultats de la régression
print(results.summary())


In [ ]:
# Utiliser les séries différenciées
diff_unemployment_series = country2.stationarity_test(country2.unemployment_series, 'Unemployment_rate')
diff_gdp_series = country2.stationarity_test(country2.gdp_series, 'Real_GDP')

# Concaténer les séries différenciées avec les retards
data = d.pd.concat([diff_unemployment_series, diff_gdp_series], axis=1)
data.columns = ['Diff_Unemployment_rate', 'Diff_Real_GDP']

# Spécifier le modèle AutoReg avec les retards
model = AutoReg(data, lags={'Diff_Unemployment_rate': lags_ur_2, 'Diff_Real_GDP': lags_gdp_2}, trend='c')
results = model.fit()

# Afficher les résultats
print(results.summary())


In [ ]:
country2_model = ARDLAnalysis(country_code1, df_unemployment, df_gdp, lags_ur_2, lags_gdp_2)
country2_model.fit_ardl_model()

Conclusion : 

### <span style="color:green">IV.2.4. Pays du cluster 3  </span><br><br>

In [ ]:
country_code3 = list_random_countries[2]
country3 = PreProcessing(country_code3, df_unemployment, df_gdp)
country3.ACF_PACF_plot()

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_3 = 7  # Lags pour Unemployment_rate
lags_gdp_3 = 0  # Lags pour Real_GDP

In [ ]:
country3_model = ARDLAnalysis(country_code3, df_unemployment, df_gdp, lags_ur_3, lags_gdp_3)
country3_model.fit_ardl_model()

### <span style="color:green">IV.2.5. Pays du cluster 4  </span><br><br>

In [ ]:
country_code4 = list_random_countries[3]
country4 = PreProcessing(country_code4, df_unemployment, df_gdp)
country4.ACF_PACF_plot()

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_4 = 7  # Lags pour Unemployment_rate
lags_gdp_4 = 0  # Lags pour Real_GDP

In [ ]:
country4_model = ARDLAnalysis(country_code4, df_unemployment, df_gdp, lags_ur_4, lags_gdp_4)
country4_model.fit_ardl_model()

Conclusion : 

# <span style="color:green">Conclusion </span><br><br>